In [1]:
def eliminar_separador_final_de_datos(filas_datos, separador):
    # Recorre cada fila de datos desde la segunda fila en adelante (debajo del header)
    nuevas_filas = []
    for fila in filas_datos:
        # Si la fila tiene el separador al final, lo eliminamos
        if fila.endswith(separador):
            fila = fila.rstrip(separador)
        nuevas_filas.append(fila)
    return nuevas_filas

def ajustar_columnas(ruta_archivo, separador):
    ajustes_realizados = True

    while ajustes_realizados:
        try:
            with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
                # Leer la primera fila (header) y todas las filas de datos
                primera_fila = archivo.readline().strip()  # Header
                filas_datos = archivo.readlines()  # Datos
                filas_datos = [fila.strip() for fila in filas_datos]  # Limpiar espacios en blanco

            # Separar el header y la primera fila de datos utilizando el separador
            columnas_header = primera_fila.split(separador)
            columnas_datos = filas_datos[0].split(separador)

            # Contar las columnas del header y de los datos
            num_columnas_header = len(columnas_header)
            num_columnas_datos = len(columnas_datos)

            # Mostrar el conteo de columnas
            print(f"Columnas en el encabezado: {num_columnas_header}")
            print(f"Columnas en los datos: {num_columnas_datos}")

            ajustes_realizados = False  # Inicializamos la variable en False

            # Caso 1: El header termina con el separador
            if primera_fila.endswith(separador):
                print("Caso: El header termina con el separador.")
                nueva_primera_fila = eliminar_separador_final(primera_fila, separador)
                primera_fila = nueva_primera_fila  # Actualizamos el header
                ajustes_realizados = True  # Indicamos que se ha realizado un ajuste

            # Caso 2: El header tiene una columna más que los datos
            elif num_columnas_header - num_columnas_datos == 1:
                print("Caso: El header tiene una columna más que los datos.")
                if not primera_fila.endswith(separador):
                    # Agregar separador al final de cada fila de datos
                    filas_datos = agregar_separador_final_a_datos(filas_datos, separador)
                    print("Se ha agregado un separador al final de cada fila de datos.")
                    ajustes_realizados = True  # Indicamos que se ha realizado un ajuste

            # Caso 3: Los datos tienen una columna más que el header
            elif num_columnas_datos - num_columnas_header == 1:
                print("Caso: Los datos tienen una columna más que el header.")
                # Eliminar separador al final de cada fila de datos si está presente
                filas_datos = eliminar_separador_final_de_datos(filas_datos, separador)
                print("Se ha eliminado el separador al final de cada fila de datos.")
                ajustes_realizados = True  # Indicamos que se ha realizado un ajuste

            # Guardar los cambios en el archivo original
            with open(ruta_archivo, 'w', encoding='utf-8') as archivo:
                archivo.write(primera_fila + '\n')  # Guardar el header
                archivo.write('\n'.join(filas_datos) + '\n')  # Guardar las filas de datos

            if ajustes_realizados:
                print("Se ha realizado un ajuste, volviendo a validar...")
            else:
                print("El archivo está correctamente ajustado.")

        except Exception as e:
            print(f"Error al procesar el archivo: {e}")
            break



In [12]:
# Ejemplo de uso:
ruta_archivo = 'datasets/ce2019_bc.csv'
separador = ','  # Define el separador del archivo CSV
ajustar_columnas(ruta_archivo, separador)

Columnas en el encabezado: 186
Columnas en los datos: 187
Caso: Los datos tienen una columna más que el header.
Se ha eliminado el separador al final de cada fila de datos.
Se ha realizado un ajuste, volviendo a validar...
Columnas en el encabezado: 186
Columnas en los datos: 186
El archivo está correctamente ajustado.


In [9]:
def validar_columnas_datos(ruta_archivo, separador, num_ajustes):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
            # Leer la primera fila (header) y todas las filas de datos
            primera_fila = archivo.readline().strip()  # Header
            filas_datos = archivo.readlines()  # Datos
            filas_datos = [fila.strip() for fila in filas_datos]  # Limpiar espacios en blanco

        # Contar la cantidad de columnas del header
        columnas_header = primera_fila.split(separador)
        num_columnas_header = len(columnas_header)
        print(f"Cantidad de columnas en el encabezado: {num_columnas_header}")

        # Validar las columnas en los datos
        inconsistencias = []  # Para guardar las filas con cantidad de columnas distintas
        conteo_columnas = {}  # Diccionario para contar cuántas filas tienen cada número de columnas
        filas_ajustadas = []  # Guardar las filas ya ajustadas

        # Contador de ajustes realizados
        ajustes_realizados = 0

        for i, fila in enumerate(filas_datos, start=2):  # Comenzamos en la línea 2 (debajo del header)
            columnas_fila = fila.split(separador)
            num_columnas_fila = len(columnas_fila)

            # Contar cuántas veces aparece un número específico de columnas
            if num_columnas_fila in conteo_columnas:
                conteo_columnas[num_columnas_fila] += 1
            else:
                conteo_columnas[num_columnas_fila] = 1

            # Registrar si hay diferencias con respecto al header
            if num_columnas_fila != num_columnas_header:
                inconsistencias.append((i, num_columnas_fila))  # Guardar el número de línea y número de columnas
            filas_ajustadas.append(columnas_fila)

        # Cálculo del promedio de columnas en las filas
        promedio_columnas = sum([len(fila.split(separador)) for fila in filas_datos]) / len(filas_datos)

        # Revisar si se necesita realizar algún ajuste
        if len(inconsistencias) <= num_ajustes:
            for i, (linea, num_columnas_fila) in enumerate(inconsistencias):
                # Caso donde hay una columna más en la fila de datos
                if num_columnas_fila == num_columnas_header + 1:
                    # Eliminar el separador al final si está presente
                    if filas_datos[linea - 2].endswith(separador):  # Revisamos las filas sin contar el header
                        filas_datos[linea - 2] = filas_datos[linea - 2].rstrip(separador)
                        ajustes_realizados += 1

                # Caso donde hay una columna menos en la fila de datos
                elif num_columnas_fila == num_columnas_header - 1:
                    # Añadir un separador al final si el promedio de columnas es mayor
                    if len(columnas_fila) < promedio_columnas:
                        filas_datos[linea - 2] += separador
                        ajustes_realizados += 1

        # Guardar los cambios en el archivo original
        with open(ruta_archivo, 'w', encoding='utf-8') as archivo:
            archivo.write(primera_fila + '\n')  # Escribir el header
            archivo.writelines(fila + '\n' for fila in filas_datos)  # Escribir las filas ajustadas

        # Mostrar reporte
        print("=== Reporte de validación de columnas ===")
        print(f"El archivo contiene {len(filas_datos)} filas de datos.")
        if len(conteo_columnas) == 1:
            print(f"Todas las filas tienen la misma cantidad de columnas: {num_columnas_header}.")
        else:
            print("Se encontraron diferencias en la cantidad de columnas entre los registros:")
            for num_columnas, cantidad in conteo_columnas.items():
                print(f" - {cantidad} filas tienen {num_columnas} columnas.")

            # Mostrar detalles de las inconsistencias
            if inconsistencias:
                print("\n=== Inconsistencias detectadas ===")
                for (linea, num_columnas_fila) in inconsistencias:
                    print(f" - Línea {linea}: tiene {num_columnas_fila} columnas.")
                print(f"Se realizaron {ajustes_realizados} ajustes.")

        print("=== Fin del reporte ===")
        
    except Exception as e:
        print(f"Error al procesar el archivo: {e}")

# Ejemplo de uso:
ruta_archivo = 'datasets/tc_codigo_actividad.csv'
separador = ','  # Define el separador del archivo CSV
num_ajustes = 5  # Número de ajustes permitidos

# Primero ajustar las columnas
ajustar_columnas(ruta_archivo, separador)


# Después validar las columnas de los datos
validar_columnas_datos(ruta_archivo, separador, num_ajustes)



Columnas en el encabezado: 3
Columnas en los datos: 4
Caso: Los datos tienen una columna más que el header.
Se ha eliminado el separador al final de cada fila de datos.
Se ha realizado un ajuste, volviendo a validar...
Columnas en el encabezado: 3
Columnas en los datos: 3
El archivo está correctamente ajustado.
Cantidad de columnas en el encabezado: 3
=== Reporte de validación de columnas ===
El archivo contiene 1847 filas de datos.
Se encontraron diferencias en la cantidad de columnas entre los registros:
 - 1522 filas tienen 3 columnas.
 - 18 filas tienen 6 columnas.
 - 231 filas tienen 4 columnas.
 - 71 filas tienen 5 columnas.
 - 5 filas tienen 7 columnas.

=== Inconsistencias detectadas ===
 - Línea 3: tiene 6 columnas.
 - Línea 8: tiene 4 columnas.
 - Línea 9: tiene 4 columnas.
 - Línea 15: tiene 5 columnas.
 - Línea 34: tiene 4 columnas.
 - Línea 44: tiene 4 columnas.
 - Línea 52: tiene 4 columnas.
 - Línea 56: tiene 7 columnas.
 - Línea 78: tiene 6 columnas.
 - Línea 79: tiene 

# cambiar delimitadores con caso especial

In [13]:
import csv
import re

def cambiar_delimitadores(input_file, output_file):
    with open(input_file, mode='r', encoding='utf-8') as infile, open(output_file, mode='w', encoding='utf-8', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)

        for row in reader:
            nueva_fila = []
            for celda in row:
                # Si la celda está entre comillas dobles, no se modifica
                if re.match(r'^".*"$', celda):
                    nueva_fila.append(celda)
                else:
                    # Reemplazar las comas por comillas simples en celdas normales
                    nueva_fila.append(celda.replace(',', "._"))
            writer.writerow(nueva_fila)

# Ejemplo de uso

#diccionario_de_datos_ce2019
#tc_codigo_actividad
input_csv = 'datasets/tc_codigo_actividad.csv'  # Archivo CSV original
output_csv = 'datasets/tc_codigo_actividad_new.csv'  # Archivo CSV modificado
cambiar_delimitadores(input_csv, output_csv)    